<h1 style="line-height:50px;;background:#9eb5db;border-radius:15px;height:50px;text-align:center;border:2px ridge;
color:white;">Ex 12</h1>

<div style="background:#9eb5db;border-radius:15px;"><h2 style="line-height:50px;height:50px;margin-left:20px;color:white">Requirements
</h2></div>

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, recall_score
from sklearn import metrics
from sklearn.cluster import KMeans
import datetime as dt

<div style="background:#9eb5db;border-radius:15px;"><h2 style="line-height:50px;height:50px;margin-left:20px;color:white">Problem
</h2></div>

In [11]:
df = pd.read_excel("Online Retail.xlsx")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [13]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
# Calculate Null and NAN Item in DF

print(df.isnull().sum().sum())
print(df.isna().sum().sum())

136534
136534


In [16]:
# Remove Row That Doesn't Have CustomerID Value 

df = df[df['CustomerID'].notnull()]

In [17]:
# Calculate Null and NAN Item in DF

print(df.isnull().sum().sum())
print(df.isna().sum().sum())

0
0


In [18]:
# Remove Post StockCode col is in row 

df = df[df["StockCode"] != 'POST']

In [19]:
# # Extracting the Date from the 'InvoiceDate' column and assigning it to 'InvoiceDay'

df['InvoiceDay'] = df['InvoiceDate'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

In [20]:
# Finding the latest invoice date and adding one day to set as the cutoff date

pin_date = max(df['InvoiceDay']) + dt.timedelta(1)

In [21]:
# Create Columns to Display All Fee Has Been Paid 

df['TotalSum'] = df['Quantity'] * df['UnitPrice']

In [22]:
# Create rfm DataFrame with Extracted Features

rfm = df.groupby('CustomerID').agg({
    'InvoiceDay': lambda x: (pin_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum' 
})

In [23]:
# Change Name of Columns in rfm DataFrame

rfm.rename(columns= {
    'InvoiceDay': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'Monetary'
}, inplace=True)

In [24]:
# Normalize & Ranking Features

r_labels = range(4, 0, -1)
r_groups, r_bins = pd.qcut(rfm['Recency'], q=4, labels=r_labels, retbins=True)
f_labels = range(1, 5)
f_groups, f_bins = pd.qcut(rfm['Frequency'], q=4, labels=f_labels, retbins=True)
m_labels = range(1, 5)
m_groups, m_bins = pd.qcut(rfm['Monetary'], q=4, labels=m_labels, retbins=True)

In [25]:
# Add Extracted Features to rfm DataFrame

rfm['R'] = r_groups.values
rfm['F'] = f_groups.values
rfm['M'] = m_groups.values

In [26]:
# Train Clustering Model

X = rfm[['R', 'F', 'M']]
kmeans = KMeans(n_clusters=10, init='k-means++', max_iter=300)
kmeans.fit(X)

KMeans(n_clusters=10)

In [27]:
# Add Kmeans_Cluster to rfm DataFrame

rfm['kmeans_cluster'] = kmeans.labels_

In [28]:
# Find Best selling product For Every 10 Cluster

Recommend_Product_Dic = {}
for i in range(10):
    Recommend_Product_Dic[f'{i}'] = []

for i in range(10):
    temp_rfm = rfm[rfm['kmeans_cluster']==i]
    temp_dic = {}
    for customer_id, _  in temp_rfm.iterrows():
        temp_df = df[df["CustomerID"]==customer_id] 
        for _, buy in temp_df.iterrows():
            if buy['StockCode'] in temp_dic:
                temp_dic[buy['StockCode']] += 1
            else:
                temp_dic[buy['StockCode']] = 1
    sorted_items = sorted(temp_dic.items(), key=lambda x: x[1], reverse=True)
    five_max_keys = [key for key, _ in sorted_items[:5]]
    for key in five_max_keys:   
        Recommend_Product_Dic[f'{i}'].append(str(key))
print(Recommend_Product_Dic)

{'0': ['85123A', '22423', '84879', '47566', '22720'], '1': ['84946', '84879', '85123A', '85099B', '22423'], '2': ['85123A', '85099B', '22423', '20725', '22197'], '3': ['21034', '22086', '85123A', '84879', '23084'], '4': ['85123A', '22423', '47566', '21034', '22960'], '5': ['85123A', '22423', '85099B', '20725', '84879'], '6': ['22423', '23084', '22086', '85123A', '84879'], '7': ['47566', '22423', '85099B', '85123A', '84879'], '8': ['22086', '84879', '85123A', '23344', '85099B'], '9': ['22423', '85123A', '47566', '22178', '22699']}


In [29]:
# Create Recommend DataFrame For All Users 

recommend_df = pd.DataFrame(columns=['CustomerID', 'RecommendStuff'])

for customer_id, row in rfm.iterrows():
    temp_df = df[df['CustomerID']==customer_id]
    all_stuff = set()
    for index, row_df in temp_df.iterrows():
        all_stuff.add(str(row_df["StockCode"]))
    
    cluster = int(row["kmeans_cluster"])
    recommend_stuff = [x for x in Recommend_Product_Dic[f'{cluster}'] if x not in list(all_stuff)]
    recommend_df = pd.concat([recommend_df, pd.DataFrame([[customer_id, recommend_stuff]], columns=['CustomerID', 'RecommendStuff'])], ignore_index=True)
print(recommend_df)

      CustomerID                         RecommendStuff
0        12346.0   [22423, 85123A, 47566, 22178, 22699]
1        12347.0         [85123A, 85099B, 20725, 22197]
2        12348.0  [47566, 22423, 85099B, 85123A, 84879]
3        12349.0          [23084, 22086, 85123A, 84879]
4        12350.0   [22423, 85123A, 47566, 22178, 22699]
...          ...                                    ...
4367     18280.0   [22423, 85123A, 47566, 22178, 22699]
4368     18281.0   [22423, 85123A, 47566, 22178, 22699]
4369     18282.0  [22086, 84879, 85123A, 23344, 85099B]
4370     18283.0                                [22423]
4371     18287.0  [85123A, 22423, 85099B, 20725, 84879]

[4372 rows x 2 columns]


In [36]:
def recommend_new_user(newUserRFM):
    input_shape = np.array(newUserRFM).ndim
    if input_shape == 1:
        new_data = np.array([newUserRFM])
        cluster = str(kmeans.predict([newUserRFM])[0])
        print(f'This New User is cluster in {kmeans.predict([newUserRFM])[0]} group and Product Recommedatoin For New User is this StockCode: \n{Recommend_Product_Dic[cluster]}')
    else:
        for sub_list in newUserRFM:
            new_data = np.array([sub_list])
            cluster = str(kmeans.predict([sub_list])[0])
            print(f'for {sub_list} New User is cluster in {kmeans.predict([sub_list])[0]} group and Product Recommedatoin For New User is this StockCode: \n{Recommend_Product_Dic[cluster]}')
            

In [37]:
# Input One New User

newUserRFM = [4,4,4]
recommend_new_user(newUserRFM)

This New User is cluster in 2 group and Product Recommedatoin For New User is this StockCode: 
['85123A', '85099B', '22423', '20725', '22197']


C:\Users\msavo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\msavo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [38]:
# Input Multi New Users

newUserRFM = [[1,1,1],[2,2,2]]
recommend_new_user(newUserRFM)

for [1, 1, 1] New User is cluster in 9 group and Product Recommedatoin For New User is this StockCode: 
['22423', '85123A', '47566', '22178', '22699']
for [2, 2, 2] New User is cluster in 4 group and Product Recommedatoin For New User is this StockCode: 
['85123A', '22423', '47566', '21034', '22960']


C:\Users\msavo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\msavo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\msavo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\msavo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
